# Animating a Projectile
#### Zain Naqavi

This code will use Vpython to animate the path of a projectile based on the equations of Classical Mechanics.

* Initially, the module functions required to create a Vpython animation are imported.
* The first section will animate the initial path of a projectile ball until it touches the ground by implementing the equations of motion.
* The second section will extend the animation of the path of a projectile ball by also implementing the conservation of momentum to calculate the bounces of the ball.

In [1]:
import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, box, label, vec

<IPython.core.display.Javascript object>

The equations of motion are implemented to calculate the path of a projectile, under the influence of gravity, launched from a height, $y_0$, at an angle, $\theta$, to the horizontal with a velocity, $v_0$.

$$ x(t) = x_0 + v_0 t \cos \theta $$

$$ y(t) = y_0 + v_0 t \sin \theta -\frac{1}{2}g t^2 $$

$$ \text{Range} = \frac{v_0^2 \sin(2\theta)}{g} $$

There is a small approximation error when the value for range is calculated using the code sine the while loop does not stop when the value for the y position of the ball is exactly zero.

In the code, the approximation for the trajectory of the ball is less accurate as the value of $\delta t$ increases since it is the increment of time over which the new position of the ball is calculated. It would be infinitely accurate if $\delta t$ were an infinitesimally small value. The value of $\delta t$ as 0.01 is used as a good approximation to the parabolic curve of projectile motion.

In [2]:
# Set up the scene
canvas(center = vector(50,20,0), range = 50)
ground = curve(pos = [vector(-5,0,0), vector(100,0,0)], color = color.green)

g = 9.8    # gravitational acceleration, ms^-2
dt = 0.01  # time interval for loop animation, s
x0 = 0.0   # initial ball x-coordinate, m

# Input initial conditions (height, angle and speed)
y0 = float(input("Input the initial height in metres: "))
dtheta = float(input("Input the initial angle in degrees: "))
theta = np.radians(dtheta)
v0 = float(input("Input the initial speed in metres/second: "))

# Initialise objects
ball = sphere(pos = vector(x0,y0,0), radius = 1, make_trail = True)
platform = box(pos = vector(x0,y0/2,0), length = 1, height = y0, width = 1, color = color.orange)

t = 0                   # initial time
while ball.pos.y >= 0:  # loop calculating position of ball as long as it is above ground
    rate(100)           # restricts animation to 100 updates per second (since dt = 0.01)
    t = t + dt          # increment value of time to calculate new position
    
    # equations of motion of ball
    ball.pos.x = x0 + v0 * t * np.cos(theta)
    ball.pos.y = y0 + v0 * t * np.sin(theta) - 0.5 * g * t ** 2
    
    
# Output final calculations
print("\nThe ball was in the air for", t, "seconds.")
print("The ball landed", ball.pos.x, "metres from the origin in the x-direction.")

if y0 == 0:  # output predicted range only if ball launched from origin (for which equation is valid)
    ball_range = np.sin(2*theta) * v0**2 / g
    range_diff = abs(ball_range - ball.pos.x)
    print("The predicted range of the ball launched is", ball_range, "metres from the origin in the x-direction.")
    print("The difference in the values for range is", range_diff, "metres.")

# Output final calculations on Vpython canvas
if y0 == 0:  # output predicted range only if ball launched from origin (for which equation is valid)
    text_calculations = 'Time in air:  {0:0.3f} s\nRange:  {1:0.3f} m\nPredicted range (<i>y</i><sub>0</sub> = 0):  {2:0.3f} m\nDifference in range values:  {3:0.3f} m'.format(t, ball.pos.x, ball_range, range_diff)
else:
    text_calculations = 'Time in air:  {0:0.3f} s\nRange:  {1:0.3f} m'.format(t, ball.pos.x)

label_calculations = label(pos = vec(ball.pos.x,-5,0), text = text_calculations, border = 4)    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Input the initial height in metres: 0
Input the initial angle in degrees: 60
Input the initial speed in metres/second: 30

The ball was in the air for 5.309999999999931 seconds.
The ball landed 79.64999999999898 metres from the origin in the x-direction.
The predicted range of the ball launched is 79.53294524550967 metres from the origin in the x-direction.
The difference in the values for range is 0.11705475448931679 metres.


### Calculating bounce
A real ball will bounce inelastically, and the normal component of the velocity is reduced by a factor $e < 1$, where $e$ is the _coefficient of restitution_. The speed and angle of the projectile will change at each bounce, depending on $e$, giving the following four equations:

The angle of incidence is defined relative to the normal (i.e. $y$-axis):

$$ \alpha = \frac{\pi}{2} - \theta $$

Thus, the angle of reflection relative to the normal is:

$$ \beta = \arctan\left(\frac{\tan \alpha }{e} \right)$$

From these the speed of the ball following the bounce, $v_0^\prime$, can be determined:

$$ v_0^\prime =  \frac{\cos \alpha}{\cos \beta} v_0 e$$

And the launch angle (relative to the $x$-axis) of the ball following the bounce:

$$ \theta^\prime = \frac{\pi}{2} - \beta $$ 

In the code, the while loop animating the bounces of the ball is stopped when the maximum height of the next bounce is negligible.

In [3]:
### Set up the scene
canvas(center = vector(50,20,0),range=50)
ground = curve(pos = [vector(-5,0,0), vector(100,0,0)], color = color.green)

g = 9.8    # gravitational acceleration, ms^-2
dt = 0.01  # time interval for loop animation, s
x0 = 0.0   # initial ball x-coordinate, m

# Input initial conditions (height, angle, speed and coefficient of restitution)
y0 = float(input("Input the initial height in metres: "))
dtheta = float(input("Input the initial angle in degrees: "))
theta = np.radians(dtheta)
v0 = float(input("Input the initial speed in metres/second: "))
e = float(input("Input the coefficient of restitution (e < 1): "))

# Initialise objects
ball = sphere(pos = vector(x0,y0,0), radius = 1, make_trail = True)
platform = box(pos = vector(x0,y0/2,0), length = 1, height = y0, width = 1, color = color.orange)

tolerance = 0.1        # tolerance for bounce height, metres
ymax = 10 * tolerance  # set bounce height arbitrarily greater than tolerance so initial ball path will always run

while ymax > tolerance:     # loop animating ball as long as the bounce height is greater than the tolerance
    t = 0                   # initialise time before each bounce
    while ball.pos.y >= 0:  # nested loop calculating position of ball for one bounce             
        rate(100)           # restricts animation to 100 updates per second (since dt = 0.01)
        t = t + dt          # increment value of time to calculate new position
    
        # equations of motion of ball
        ball.pos.x = x0 + v0 * t * np.cos(theta)
        ball.pos.y = y0 + v0 * t * np.sin(theta) - 0.5 * g * t ** 2
    
    ball.pos.y = 0  # fix approximation error from equations of motion so nested loop will still run for bounce
    
    # new initial conditions for bounce   
    y0 = ball.pos.y
    x0 = ball.pos.x
    alpha = np.pi/2 - theta                 # angle of incidence, radians
    beta = np.arctan(np.tan(alpha)/e)       # angle of reflection, radians
    theta = np.pi/2 - beta
    v0 = v0 * e * np.cos(alpha) / np.cos(beta)
    
    ymax = 0.5 * (v0 * np.sin(theta))**2 / g  # new maximum height of bounce

<IPython.core.display.Javascript object>

Input the initial height in metres: 5
Input the initial angle in degrees: 60
Input the initial speed in metres/second: 15
Input the coefficient of restitution (e < 1): 0.7


The code now gives a more realistic result of the motion of a ball since the height of each new bounce decreases which is a more accurate model since a real ball will lose energy as it collides with the ground. The range of each new bounce also decreases for the same reason. However, there will be an infinite number of bounces before the ball comes to a complete stop (in a vacuum which the model assumes) so the code is still an approximation of the complete motion of the ball.